In [25]:
import h5py
import pandas as pd
import numpy as np
import tables


In [26]:
import h5py

f = h5py.File('SVHN_single_grey1.h5')

In [3]:
#df = np.array(h5py.File('SVHN_single_grey1.h5'))

In [27]:
for key in f.keys():
    print(key) #Names of the groups in HDF5 file.

X_test
X_train
X_val
y_test
y_train
y_val


In [28]:
X_train = f['X_train'][:]
X_test = f['X_test'][:]
X_val = f['X_val'][:]
y_test = f['y_test'][:]
y_train = f['y_train'][:]
y_val = f['y_val'][:]

In [29]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [30]:
X_test.dtype
X_train.dtype
X_val.dtype
y_test.dtype
y_train.dtype
y_val.dtype

dtype('<f4')

dtype('<f4')

dtype('<f4')

dtype('uint8')

dtype('uint8')

dtype('uint8')

In [31]:
list(f.keys())

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [32]:
X_train.shape
X_test.shape
X_val.shape
y_train.shape
y_test.shape
y_val.shape

(42000, 32, 32)

(18000, 32, 32)

(60000, 32, 32)

(42000,)

(18000,)

(60000,)

In [9]:
X_train_ml = X_train.reshape(X_train.shape[0], 32*32).astype('float32')
X_test_ml = X_test.reshape(X_test.shape[0], 32*32).astype('float32')
X_val_ml = X_val.reshape(X_val.shape[0], 32*32).astype('float32')

In [10]:
X_train_ml.shape
X_test_ml.shape
X_val_ml.shape
y_train.shape
y_test.shape
y_val.shape

(42000, 1024)

(18000, 1024)

(60000, 1024)

(42000,)

(18000,)

(60000,)

In [33]:
num_classes = np.unique(y_train).shape[0]
num_classes

10

In [12]:
print(y_train.shape)

(42000,)


In [34]:
#Similar to one hot encoding, here it converts one number to ten numbers
import keras
trainY = keras.utils.to_categorical(y_train, num_classes=10)
testY = keras.utils.to_categorical(y_test, num_classes=10)
valY = keras.utils.to_categorical(y_val, num_classes=10)

In [14]:
print(trainY.shape)

(42000, 10)


In [16]:
#Implement and apply an optimal k-Nearest Neighbor (kNN) classifier (7.5 points) 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
NNH = KNeighborsClassifier(n_neighbors= 3)

In [17]:
NNH.fit(X_train_ml, trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [18]:
#Print the classification metric report (2.5 points) 
predictions = NNH.predict(X_val_ml)
score = NNH.score(X_val_ml, valY)
print(score)

0.5349666666666667


In [20]:
#Print the classification metric report (2.5 points) 
predictions = NNH.predict(X_test_ml)
score = NNH.score(X_test_ml, testY)
print(score)

0.3783888888888889


In [35]:
#Initialize model, reshape & normalize data
import keras
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Reshape((1024,),input_shape=(32,32,)))
model.add(keras.layers.BatchNormalization())

In [36]:
#Add 1st hidden layer
model.add(keras.layers.Dense(200, activation='relu'))

In [37]:
#Add 2nd hidden layer
model.add(keras.layers.Dense(100, activation='relu'))

In [38]:
#Add 3rd hidden layer
model.add(keras.layers.Dense(60, activation='relu'))

In [39]:
#Add 4th hidden layer
model.add(keras.layers.Dense(30, activation='relu'))

In [40]:
#Add OUTPUT layer
model.add(keras.layers.Dense(10, activation='softmax'))

In [41]:
# Loss and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               205000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_3 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                310       
Total para

In [43]:
print(X_train_ml.shape)
print(trainY.shape)
print(X_val_ml.shape)
print(valY.shape)

(42000, 1024)
(42000, 10)
(60000, 1024)
(60000, 10)


In [44]:
# Train the model
model.fit(X_train, trainY, batch_size=100, nb_epoch=20, 
validation_data=(X_val, valY))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 42000 samples, validate on 60000 samples
Epoch 1/20
42000/42000 [==============================] - 8s 196us/step - loss: 2.2228 - acc: 0.1859 - val_loss: 2.0645 - val_acc: 0.3095
Epoch 2/20
42000/42000 [==============================] - 8s 188us/step - loss: 1.7982 - acc: 0.4077 - val_loss: 1.5015 - val_acc: 0.5188
Epoch 3/20
42000/42000 [==============================] - 8s 186us/step - loss: 1.3605 - acc: 0.5695 - val_loss: 1.1828 - val_acc: 0.6419
Epoch 4/20
42000/42000 [==============================] - 8s 183us/step - loss: 1.1444 - acc: 0.6497 - val_loss: 1.0283 - val_acc: 0.6900
Epoch 5/20
42000/42000 [==============================] - 8s 202us/step - loss: 1.0117 - acc: 0.6901 - val_loss: 0.9175 - val_acc: 0.7259
Epoch 6/20
42000/42000 [==============================] - 8s 186us/step - loss: 0.9185 - acc: 0.7225 - val_loss: 0.8568 - val_acc: 0.7410
Epoch 7/20
42000/42000 [==============================] - 8s 192us/step - loss: 0.8567 - acc: 0.7404 - val_loss: 0.8017 - 

In [46]:
# Train the model
model.fit(X_train, trainY, batch_size=100, nb_epoch=20, 
validation_data=(X_test, testY))

Train on 42000 samples, validate on 18000 samples
Epoch 1/20
 1800/42000 [>.............................] - ETA: 3s - loss: 0.3528 - acc: 0.8817

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


42000/42000 [==============================] - 6s 135us/step - loss: 0.3491 - acc: 0.8900 - val_loss: 0.6430 - val_acc: 0.8291
Epoch 2/20
42000/42000 [==============================] - 6s 141us/step - loss: 0.3399 - acc: 0.8931 - val_loss: 0.6441 - val_acc: 0.8301
Epoch 3/20
42000/42000 [==============================] - 6s 137us/step - loss: 0.3339 - acc: 0.8953 - val_loss: 0.5974 - val_acc: 0.8413
Epoch 4/20
42000/42000 [==============================] - 6s 144us/step - loss: 0.3335 - acc: 0.8954 - val_loss: 0.6006 - val_acc: 0.8427
Epoch 5/20
42000/42000 [==============================] - 5s 125us/step - loss: 0.3244 - acc: 0.8980 - val_loss: 0.6333 - val_acc: 0.8333
Epoch 6/20
42000/42000 [==============================] - 6s 133us/step - loss: 0.3257 - acc: 0.8991 - val_loss: 0.6391 - val_acc: 0.8349
Epoch 7/20
42000/42000 [==============================] - 6s 143us/step - loss: 0.3203 - acc: 0.8987 - val_loss: 0.5994 - val_acc: 0.8442
Epoch 8/20
42000/42000 [=====================